In [8]:
from gurobipy import *

gas_sales = [70,60,50]
crude_purchases = [45,35,25]

gas_constrations = [3000,2000,1000]
crude_constrations = 5000

crude_octan_rating = [12,6,8]

octane_level_constraints = [
    [2, -4, -2],
    [0, 4, -2],
    [6,0,2]    
]

x_variables_names = [["x"+str(i)+str(j) for i in range(3)] for j in range(3)]
ad_variables_names = ["a"+str(i) for i in range(3)]

# print(x_variables_names)
# print(ad_variables_names)

In [ ]:
m = Model("Blending Problems")

x_variables = []
ad_variables = []

for i in range(3):
    x_variables.append([])
    for j in range(3):
        x_variables[i].append(
            m.addVar(
            vtype = GRB.CONTINUOUS,
            obj = gas_sales[j]-crude_purchases[i]-4, 
            name="(%s)" % (x_variables_names[i][j]))
        )
        
for j in range(3):
    ad_variables.append(
        m.addVar(
        vtype = GRB.CONTINUOUS,
        obj = -1,
        name="(%s)"% (ad_variables_names[j]))
    )
    
m.modelSense = GRB.MAXIMIZE
m.update()

m.getObjective()

In [ ]:
#Gas Constraints
for gas in range(3):
    constraions = [x_variables[crude][gas] for crude in range(3)]
    constraions.append(-10 * ad_variables[gas])
    m.addConstr(
        quicksum(constraions
                ) == gas_constrations[gas],
        "Gas %d requirement" % gas)
    
m.update()
m.getConstrs()

In [ ]:
for crude in range(3):
    constraions = [x_variables[crude][gas] for gas in range(3)]
    m.addConstr(
        quicksum(constraions
                ) <= crude_constrations,
        "Crude Oil %d requirement" % crude
    )
    
m.update()
m.getConstrs()    

In [ ]:
# Constraint for refinery capacity limit
constraions = [x_variables[crude][gas] for crude in range(3) for gas in range(3)]
m.addConstr(
    quicksum(constraions) == 14000, "reginery capacity limit"
)

m.update()
m.getConstrs()

In [ ]:
sulfur_content = [0.005, 0.02, 0.03]
sulfur_at_most = [0.01,0.02,0.01]

for gas in range(3):
    constrains_lhs = [sulfur_content[crude]*x_variables[crude][gas] for crude in range(3)]
    
    constrains_rhs = [sulfur_at_most[crude]*x_variables[crude][gas] for crude in range(3)]
    
    m.addConstr(
        quicksum(constrains_lhs) >= quicksum(constrains_rhs),
        "Sulfur content in Gas %d requirment" % gas
    )
    
m.update()
m.getConstrs()
m.optimize()

In [21]:
for v in m.getVars():
    print(v.varName,v.x)
    
print(m.getObjective().getValue())    

(x00) 4000.0
(x10) 0.0
(x20) 0.0
(x01) 2000.0
(x11) 2000.0
(x21) 1000.0
(x02) 5000.0
(x12) 0.0
(x22) 0.0
(a0) 800.0
(a1) 0.0
(a2) 0.0
403200.0
